**Задача**: ответ на да/нет вопрос на основе отрывка (бинарная классификация). **Язык**: русский. **Датасет**: https://huggingface.co/datasets/d0rj/boolq-ru.


#Установка пакетов и скачивание данных

In [ ]:
!pip install --upgrade accelerate
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from datasets import load_dataset
dataset = load_dataset("d0rj/boolq-ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [ ]:
train_data = dataset["train"]
dev_data = dataset["validation"]
print(train_data[0])
print(dev_data[0])

{'question': 'говорят ли иран и афганистан на одном языке', 'answer': True, 'passage': 'Персидский (/ˈpɜːrʒən, -ʃən/), также известный под своим эндонимом фарси (فارسی фарси (fɒːɾˈsiː) (слушать)), является одним из западноиранских языков в индоиранской ветви индоевропейской языковой семьи. На нем в основном говорят в Иране, Афганистане (официально известном как дари с 1958 года) и Таджикистане (официально известном как таджики с советских времен), а также в некоторых других регионах, которые исторически были персидскими обществами и считались частью Большого Ирана. Он написан персидским алфавитом, модифицированным вариантом арабского письма, которое в свою очередь произошло от арамейского алфавита.'}
{'question': 'этанол требует больше энергии, что производит', 'answer': False, 'passage': 'Вся биомасса проходит по крайней мере некоторые из этих этапов: ее нужно выращивать, собирать, сушить, ферментировать, перегонять и сжигать. Все эти шаги требуют ресурсов и инфраструктуры. Общее коли

Таким образом, в датасете есть отрывок, вопрос и ответ на этот вопрос на основании отрывка - True/False.

#Обучение с помощью встроенного Trainer

Сначала посмотрим, как справится с задачей модель, обученная с помощью Trainer.

Будем использовать модель "DeepPavlov/rubert-base-cased" и соответствующий токенизатор.

In [ ]:
from transformers import AutoTokenizer

def make_dataset(data, tokenizer, label_field='answer'):
    answer = []
    for i, item in enumerate(data):
        result = tokenizer(item["question"], item["passage"], max_length=512, truncation=True)
        if label_field in item:
            result["labels"] = int(item["answer"])
        answer.append(result)
    return answer

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

train_dataset = make_dataset(train_data, tokenizer)
dev_dataset = make_dataset(dev_data, tokenizer)
for key, value in train_dataset[0].items():
    print(key, value)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

input_ids [101, 20034, 11089, 18792, 851, 27500, 11850, 1469, 11463, 11956, 102, 98799, 120, 134, 100, 128, 130, 600, 588, 259, 134, 122, 128, 3060, 13596, 2068, 9910, 20700, 14106, 1455, 106390, 120, 1195, 12396, 1182, 1261, 106390, 120, 100, 122, 120, 52207, 122, 122, 128, 4415, 8082, 1703, 29033, 58525, 2999, 14099, 845, 61003, 58525, 2072, 26327, 119325, 43684, 12060, 132, 2988, 4757, 845, 10171, 20034, 845, 31216, 128, 24116, 120, 13237, 48170, 2739, 12332, 852, 869, 11459, 1768, 122, 851, 49431, 120, 13237, 48170, 2739, 41123, 852, 869, 16186, 20162, 122, 128, 625, 3060, 845, 10674, 6166, 21118, 128, 4427, 37064, 3294, 95796, 852, 117721, 851, 36202, 11917, 19461, 19422, 132, 4941, 29307, 95796, 34048, 1455, 128, 34079, 21437, 46713, 33057, 19202, 128, 8956, 845, 6297, 10843, 12655, 1641, 67739, 27487, 39409, 132, 102]
token_type_ids [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred_labels = np.argmax(logits, axis=-1)
    are_equal = (pred_labels == labels)
    correct, total = np.sum(are_equal), len(labels)
    return {
        "Accuracy": 100 * correct / total,
        "P": precision_score(labels, pred_labels, average='macro') * 100,
        "R": recall_score(labels, pred_labels, average='macro') * 100,
        'F1': f1_score(labels, pred_labels, average='macro') * 100
    }

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.optimization import AdamW

import numpy as np

model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=4,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=4, warmup_ratio=0.1,
                                  gradient_accumulation_steps=4,
                                  metric_for_best_model="Accuracy")
trainer = Trainer(
    model=model,
    optimizers=(optimizer, None), # optimizers=(optimizer, scheduler)
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # добавляет паддинг в более короткие элементы батча
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,P,R,F1
0,0.659000,0.630629,65.565749,65.879380,56.306672,52.975695
1,0.580400,0.578180,68.990826,67.074067,67.148397,67.109847
2,0.483100,0.610661,70.428135,68.660204,68.889893,68.762926
3,0.400300,0.657222,70.856269,69.212946,69.645692,69.384342


TrainOutput(global_step=2356, training_loss=0.5016975564746177, metrics={'train_runtime': 1883.814, 'train_samples_per_second': 20.017, 'train_steps_per_second': 1.251, 'total_flos': 3768502645929120.0, 'train_loss': 0.5016975564746177, 'epoch': 3.9983029274501485})

Измерим качество на валидационном датасете.

In [ ]:
test_dataset = make_dataset(dev_data, tokenizer)
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.0741987 ,  1.2157439 ],
       [-0.21760738,  0.22144948],
       [ 0.71808815, -0.37568602],
       ...,
       [-1.7464733 ,  1.7647263 ],
       [ 1.0785282 , -0.7644194 ],
       [ 1.1070418 , -0.727165  ]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 1, 0]), metrics={'test_loss': 0.6572222113609314, 'test_Accuracy': 70.85626911314985, 'test_P': 69.21294639940515, 'test_R': 69.64569247672101, 'test_F1': 69.38434164854814, 'test_runtime': 54.8803, 'test_samples_per_second': 59.584, 'test_steps_per_second': 7.453})

Метрики (Accuracy = 70.8, F-мера = 69.3) не слишком высокие, но приемлемые.

#Протестируем модель на новых примерах

In [ ]:
data_samples = [
    {
        "question": "в 1961 году был пилотируемый полет в космос",
        "passage": "Полет Юрия Гагарина в космос состоялся 12 апреля 1961 года. Гагарин стал первым человеком в истории, который совершил полет в космос. Он находился на борту корабля Восток-1 и совершил один оборот вокруг Земли, что заняло примерно 108 минут. Полет Юрия Гагарина считается важным событием в истории освоения космоса и ознаменовал начало эры пилотируемых космических полетов."
        #"answer": True
    },
    {
        "question": "Пенициллин получают из грибов рода Penicillium",
        "passage": "Пенициллины — антимикробные препараты, относящиеся к классу β-лактамных антибиотиков. Были открыты Александром Флемингом в 1928 году. Природным продуцентом пенициллинов являются грибы рода Penicillium, наиболее активный из природных пенициллинов — бензилпенициллин, применяющийся с конца 1940-х годов."
        #"answer": True
    },
    {
       "question": "в сентябре 2015 года был прием заявок на Летние Олимпийские игры",
       "passage": "Летние Олимпийские игры 2024 — 33-е по счёту летние Олимпийские игры, которые планируется провести с 26 июля по 11 августа 2024 года. Приём официальных заявок начался 15 января 2015 года и был закончен 15 сентября 2015 года. 13 сентября 2017 года МОК на своей 131-й сессии в Лиме официально объявил столицу Олимпийских игр — Париж."
        #"answer": True
    },
    {
        "question": "Собо́р Свято́й Софи́и был православным собором",
        "passage": "Собо́р Свято́й Софи́и — памятник архитектуры в Турции. Всемирно известный памятник византийского зодчества, символ «золотого века» Византии. Самое величественное здание юстиниановской эпохи, храм построен в византийской столице в 532—537 годах. Ныне бывший патриарший православный собор, находящийся в историческом центре современного Стамбула (ранее Константинополь) в районе Султанахмет."
        #"answer": True
    },
    {
        "question": "мхи отличаются от сосудистых растений",
        "passage": "У мхов нет цветков, корней и проводящей системы. Размножаются мхи спорами, которые созревают в спорангиях на спорофите. В жизненном цикле, в отличие от сосудистых растений, преобладает гаплоидный (то есть с одинарным набором непарных хромосом) гаметофит (половое поколение)."
        #"answer": True
    },
    {
        "question": "Диккенс женился на сестре Мэри",
        "passage": "2 апреля 1836 года Чарльз Диккенс женился на Кэтрин Томсон Хогарт (19 мая 1815 — 22 ноября 1879), старшей дочери его приятеля, журналиста Джорджа Хогарта. Диккенс был очень привязан к её сестре Мэри, ранняя смерть которой потрясла его. Другая сестра Кэтрин, Джорджина Хогарт, жила с четой Диккенсов и воспитывала их детей."
        #"answer": True
    },
    {
        "question": "Диккенс женился на Эллен Тернан",
        "passage": "В 1857 году Чарльз встретил 18-летнюю актрису Эллен Тернан и сразу влюбился. Снял для неё квартиру, долгие годы навещал свою любовь, впоследствии включил её в завещание. В 1858 году Диккенс развёлся с Кэтрин; роман с Элен продлился до смерти писателя. На сцену она больше не вышла. Этим близким отношениям посвящён художественный фильм «Невидимая женщина»."
        #"answer": False
    },
    {
        "question": "Системы здравоохранения легко справляются с пандемией",
        "passage": "Пандемия опасна тем, что одновременное заболевание инфекцией множества людей может привести к перегруженности системы здравоохранения с повышенным количеством госпитализаций и летальных исходов. Системы здравоохранения могут оказаться не готовы к необычайно большому количеству тяжелобольных пациентов"
        #"answer": False
    },

    {
        "question": "Чтобы производить кока-колу конртакт с компанией не нужен",
        "passage": "Компания Coca-Cola производит концентрат для кока-колы, который затем продаётся лицензированным разливщикам Coca-Cola по всему миру. Разливщики, которые имеют эксклюзивные контракты с компанией на территории, производят готовый продукт в банках и бутылках из концентрата в сочетании с фильтрованной водой и подсластителями.Типичная банка объёмом 330 мл содержит 38 граммов сахара."
        #"answer": False
    },

    {
        "question": "На официальных играх дизайн шахматных фигур может быть разным",
        "passage": "Дизайн шахматных фигур разнообразен. Стандартном де-факто стал так называемый стаунтоновский комплект, разработанный в 1849 году и по сей день применяемый в официальных шахматных турнирах, но существует множество резко отличающихся вариантов дизайна, например, фигуры, изображающие солдат конкретных армий, известных людей, сказочных персонажей, животных, персонажей определённых произведений и так далее."
        #"answer": False
    }
]
check_dataset = make_dataset(data_samples, tokenizer)

In [ ]:
import scipy

def predict_with_trainer(trainer, dataset, classes):
    predictions = trainer.predict(dataset)
    probs = scipy.special.softmax(predictions.predictions, axis=-1)
    answer = [{"label": classes[np.argmax(elem)], "probs": elem} for elem in probs]
    return answer

predictions = predict_with_trainer(trainer, check_dataset, classes=[True, False])
for sample, elem in zip(data_samples, predictions):
    print(sample["question"])
    print(sample["passage"])
    print(elem)

в 1961 году был пилотируемый полет в космос
Полет Юрия Гагарина в космос состоялся 12 апреля 1961 года. Гагарин стал первым человеком в истории, который совершил полет в космос. Он находился на борту корабля Восток-1 и совершил один оборот вокруг Земли, что заняло примерно 108 минут. Полет Юрия Гагарина считается важным событием в истории освоения космоса и ознаменовал начало эры пилотируемых космических полетов.
{'label': False, 'probs': array([0.02768104, 0.97231895], dtype=float32)}
Пенициллин получают из грибов рода Penicillium
Пенициллины — антимикробные препараты, относящиеся к классу β-лактамных антибиотиков. Были открыты Александром Флемингом в 1928 году. Природным продуцентом пенициллинов являются грибы рода Penicillium, наиболее активный из природных пенициллинов — бензилпенициллин, применяющийся с конца 1940-х годов.
{'label': False, 'probs': array([0.02968831, 0.97031164], dtype=float32)}
в сентябре 2015 года был прием заявок на Летние Олимпийские игры
Летние Олимпийские иг

Как видно, модель справляется с заданием плохо. Верные предсказания получились в 3 из 10 примеров. Это может говорить о том, что для данной задачи требуется другой подход.

#Рукописная модель

Теперь посмотрим, как справится с заданием рукописная модель.

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader

def batch_collate_fn(samples, pad_value=0, dtype=np.int64, device="cuda"):
    L = max(len(elem["input_ids"]) for elem in samples)
    padded_input_ids = np.stack([
        np.concatenate([elem["input_ids"], np.full(shape=(L-len(elem["input_ids"]),), fill_value=pad_value, dtype=dtype)])
        for elem in samples
    ])
    padded_attention_mask = np.stack([
        np.concatenate([
            elem["attention_mask"],
            np.zeros(shape=(L-len(elem["attention_mask"]),), dtype=dtype)
        ]) for elem in samples
    ])
    labels = np.stack([elem["labels"] for elem in samples])
    return {"index": np.array([elem["index"] for elem in samples]),
            "input_ids": torch.LongTensor(padded_input_ids).to(device),
            "attention_mask": torch.LongTensor(padded_attention_mask).to(device),
            "labels": torch.LongTensor(labels).to(device)}

In [ ]:
from transformers import AutoTokenizer

def make_dataset(data, tokenizer, label_field='answer'):
    answer = []
    for i, item in enumerate(data):
        result = tokenizer(item["question"], item["passage"], max_length=512, truncation=True)
        if label_field in item:
            result["labels"] = int(item["answer"])
        result["index"] = i
        answer.append(result)
    return answer

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

train_dataset = make_dataset(train_data, tokenizer)
dev_dataset = make_dataset(dev_data, tokenizer)
for key, value in train_dataset[0].items():
    print(key, value)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

input_ids [101, 20034, 11089, 18792, 851, 27500, 11850, 1469, 11463, 11956, 102, 98799, 120, 134, 100, 128, 130, 600, 588, 259, 134, 122, 128, 3060, 13596, 2068, 9910, 20700, 14106, 1455, 106390, 120, 1195, 12396, 1182, 1261, 106390, 120, 100, 122, 120, 52207, 122, 122, 128, 4415, 8082, 1703, 29033, 58525, 2999, 14099, 845, 61003, 58525, 2072, 26327, 119325, 43684, 12060, 132, 2988, 4757, 845, 10171, 20034, 845, 31216, 128, 24116, 120, 13237, 48170, 2739, 12332, 852, 869, 11459, 1768, 122, 851, 49431, 120, 13237, 48170, 2739, 41123, 852, 869, 16186, 20162, 122, 128, 625, 3060, 845, 10674, 6166, 21118, 128, 4427, 37064, 3294, 95796, 852, 117721, 851, 36202, 11917, 19461, 19422, 132, 4941, 29307, 95796, 34048, 1455, 128, 34079, 21437, 46713, 33057, 19202, 128, 8956, 845, 6297, 10843, 12655, 1641, 67739, 27487, 39409, 132, 102]
token_type_ids [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
from functools import partial

collate_fn = partial(batch_collate_fn, pad_value=tokenizer.pad_token_id)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
for batch in train_dataloader:
    for key, value in batch.items():
        print(key, value.shape, end=" ")
    print("")
    break

index (8,) input_ids torch.Size([8, 265]) attention_mask torch.Size([8, 265]) labels torch.Size([8]) 


In [ ]:
from transformers import AutoModel

pavlov_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
with torch.no_grad():
    batch_output = pavlov_model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
for key, value in batch_output.items():
    print(key, value.shape)

last_hidden_state torch.Size([8, 265, 768])
pooler_output torch.Size([8, 768])


In [ ]:
import torch.nn as nn
from transformers.optimization import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

class BasicTransformersClassificationModel(nn.Module):

    def __init__(self, model, labels_number, grad_accum=1, lr=1e-5, warmup=0.1, device="cpu", **kwargs):
        super(BasicTransformersClassificationModel, self).__init__()
        self.model = model
        self.labels_number = labels_number
        self.grad_accum = grad_accum
        self.warmup = warmup
        self.build_network(labels_number)
        # определяем функцию потерь
        self.log_softmax = nn.LogSoftmax(dim=-1)
        self.criterion = nn.NLLLoss(reduction="mean")
        self.device = device
        if self.device is not None:
            self.to(self.device)
        self.optimizer = AdamW(self.parameters(), lr=lr, weight_decay=0.01)
        self.scheduler = None
        self.step = 0

    @property
    def hidden_size(self):
        return self.model.config.hidden_size

    def forward(self, input_ids, **kwargs):
        raise NotImplementedError("You should implement forward pass in your derived class.")

    def set_scheduler(self, num_train_steps):
        num_warmup_steps = int(self.warmup*num_train_steps)
        self.scheduler = get_linear_schedule_with_warmup(self.optimizer,
                                                         num_warmup_steps=num_warmup_steps,
                                                         num_training_steps=num_train_steps
                                                         )

    def train_on_batch(self, input_ids, labels, **kwargs):
        self.train()
        if self.step % self.grad_accum == 0: # grad_accum=4 -> 4,8,12,...
            self.optimizer.zero_grad() # обнуление градиента
        batch_output = self._validate(input_ids, labels, **kwargs)
        batch_output["loss"].backward() # вычисление градиентного спуска
        if (self.step+1) % self.grad_accum == 0: # grad_accum=4 -> 3,7,11,...
            self.optimizer.step() # изменение параметров модели
            if self.scheduler is not None:
                self.scheduler.step()
        self.step += 1
        return batch_output

    def validate_on_batch(self, input_ids, labels, **kwargs):
        self.eval()
        with torch.no_grad():
            return self._validate(input_ids, labels, **kwargs)

    def _validate(self, input_ids, labels, **kwargs):
        if self.device is not None:
            input_ids, labels = input_ids.to(self.device), labels.to(self.device)
        batch_output = self(input_ids, **kwargs) #   self.forward(x) = self.__call__(x)
        loss = self.criterion(batch_output["log_probs"], labels)
        batch_output["loss"] = loss
        return batch_output

In [ ]:
class TransformersClassificationModel(BasicTransformersClassificationModel):

    def build_network(self, labels_number):
        self.proj_layer = torch.nn.Linear(self.hidden_size, self.labels_number)
        return self

    def forward(self, input_ids, attention_mask=None, **kwargs):
        input_ids = input_ids.to(self.device)
        cls_output = self.model(input_ids, attention_mask)["pooler_output"]
        logits = self.proj_layer(cls_output)
        log_probs = self.log_softmax(logits)
        _, labels = torch.max(log_probs, dim=-1)
        return {"log_probs": log_probs, "labels": labels}

In [ ]:
model = TransformersClassificationModel(pavlov_model, labels_number=2, device="cuda")
for i in range(100):
    loss = model.train_on_batch(**batch)["loss"].item()
    if i < 5 or (i+1) % 10 == 0:
        print(i, loss)
print(model.validate_on_batch(**batch)["loss"].item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.7049118280410767
1 0.6872738003730774
2 0.6671721339225769
3 0.6188716292381287
4 0.6094803214073181
9 0.5249603390693665
19 0.24584315717220306
29 0.06734034419059753
39 0.023923110216856003
49 0.013878203928470612
59 0.008307632990181446
69 0.005836912430822849
79 0.004841167945414782
89 0.003970632795244455
99 0.0034485883079469204
0.002591966185718775


In [ ]:
from tqdm.auto import tqdm

def update_metrics(metrics, batch_output, batch_labels):
    n_batches = metrics["n_batches"]
    # пересчитываем средний штраф
    metrics["loss"] = (metrics["loss"] * n_batches + batch_output["loss"].item()) / (n_batches + 1)
    metrics["n_batches"] += 1
    are_equal = (batch_output["labels"] == batch_labels).cpu().numpy().astype("int")
    batch_labels = batch_labels.cpu().numpy().astype("int")
    are_TP = are_equal * batch_labels
    are_FP = (1.0 - are_equal) * (1-batch_labels)
    are_FN = (1.0 - are_equal) * batch_labels
    metrics["correct"] += are_equal.sum()
    metrics["total"] += are_equal.shape[0]
    metrics["TP"] += are_TP.sum()
    metrics["FP"] += are_FP.sum()
    metrics["FN"] += are_FN.sum()
    metrics["accuracy"] = metrics["correct"] / max(metrics["total"], 1)
    metrics["P"] = metrics["TP"] / max(metrics["TP"]+metrics["FP"], 1)
    metrics["R"] = metrics["TP"] / max(metrics["TP"]+metrics["FN"], 1)
    metrics["F1"] = metrics["TP"] / max(metrics["TP"]+0.5*metrics["FN"]+0.5*metrics["FP"], 1)

def do_epoch(model, dataloader, mode="validate", epoch=1):
    metrics = {"correct": 0, "total": 0, "loss": 0.0, "n_batches": 0,
               "TP": 0, "FP": 0, "FN": 0}
    func = model.train_on_batch if mode == "train" else model.validate_on_batch
    progress_bar = tqdm(dataloader, leave=True)
    progress_bar.set_description(f"{mode}, epoch={epoch}")
    for batch in progress_bar:
        batch_output = func(**batch)
        update_metrics(metrics, batch_output, batch["labels"])
        progress_bar.set_postfix({"loss": round(metrics["loss"], 4), "acc": round(100 * metrics["accuracy"], 2),
                                  "P": round(100 * metrics["P"], 2), "R": round(100 * metrics["R"], 2),
                                  "F1": round(100 * metrics["F1"], 2)})
    return metrics

In [ ]:
BATCH_SIZE = 16

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

Обучение

In [ ]:
import math
NEPOCHS = 5

GRAD_ACCUM = 1

pavlov_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to("cuda")
model = TransformersClassificationModel(model=pavlov_model, labels_number=2, grad_accum=GRAD_ACCUM, device="cuda", lr=1e-5)
num_train_steps = int(math.ceil(len(train_dataset) / (GRAD_ACCUM * BATCH_SIZE)))*NEPOCHS
model.set_scheduler(num_train_steps=num_train_steps)

best_val_acc = 0.0
checkpoint = "checkpoint_best.pt"
for epoch in range(NEPOCHS):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)
model.load_state_dict(torch.load(checkpoint))
do_epoch(model, test_dataloader, mode="validate", epoch="evaluate")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/590 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/590 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

...

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/590 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

{'correct': 2364,
 'total': 3270,
 'loss': 0.6665034904712583,
 'n_batches': 205,
 'TP': 1679,
 'FP': 552.0,
 'FN': 354.0,
 'accuracy': 0.7229357798165138,
 'P': 0.7525773195876289,
 'R': 0.8258730939498279,
 'F1': 0.7875234521575984}

#Измерим качество на валидационном датасете

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def predict_with_model(model, X, batch_size=8):
    model.eval()
    dataloader = DataLoader(X, batch_size=batch_size, collate_fn=collate_fn)
    answer = [None] * len(X)
    for batch in dataloader:
        with torch.no_grad():
            batch_answer = model(**batch)
        for i, label in zip(batch["index"], batch_answer["labels"]):
            answer[i] = label.item()
    return answer

test_dataset = dev_dataset
y_pred = predict_with_model(model, test_dataset)
test_labels = [x['labels'] for x in test_dataset]
print(" ".join(
    f"{100*x:.2f}" for x in f1_score(test_labels, y_pred, average=None)
))
print(f"{100*f1_score(test_labels, y_pred, average='macro'):.2f} {100*accuracy_score(test_labels, y_pred):.2f}")

60.19 78.75
69.47 72.29


Как видно, значения F-меры и точности не слишком высокие, но приемлемые.

#Протестируем модель на новых примерах

In [ ]:
data_samples = [
    {
        "question": "в 1961 году был пилотируемый полет в космос",
        "passage": "Полет Юрия Гагарина в космос состоялся 12 апреля 1961 года. Гагарин стал первым человеком в истории, который совершил полет в космос. Он находился на борту корабля Восток-1 и совершил один оборот вокруг Земли, что заняло примерно 108 минут. Полет Юрия Гагарина считается важным событием в истории освоения космоса и ознаменовал начало эры пилотируемых космических полетов.",
        "answer": True
    },
    {
        "question": "Пенициллин получают из грибов рода Penicillium",
        "passage": "Пенициллины — антимикробные препараты, относящиеся к классу β-лактамных антибиотиков. Были открыты Александром Флемингом в 1928 году. Природным продуцентом пенициллинов являются грибы рода Penicillium, наиболее активный из природных пенициллинов — бензилпенициллин, применяющийся с конца 1940-х годов.",
        "answer": True
    },
    {
       "question": "в сентябре 2015 года был прием заявок на Летние Олимпийские игры",
       "passage": "Летние Олимпийские игры 2024 — 33-е по счёту летние Олимпийские игры, которые планируется провести с 26 июля по 11 августа 2024 года. Приём официальных заявок начался 15 января 2015 года и был закончен 15 сентября 2015 года. 13 сентября 2017 года МОК на своей 131-й сессии в Лиме официально объявил столицу Олимпийских игр — Париж.",
        "answer": True
    },
    {
        "question": "Собо́р Свято́й Софи́и был православным собором",
        "passage": "Собо́р Свято́й Софи́и — памятник архитектуры в Турции. Всемирно известный памятник византийского зодчества, символ «золотого века» Византии. Самое величественное здание юстиниановской эпохи, храм построен в византийской столице в 532—537 годах. Ныне бывший патриарший православный собор, находящийся в историческом центре современного Стамбула (ранее Константинополь) в районе Султанахмет.",
        "answer": True
    },
    {
        "question": "мхи отличаются от сосудистых растений",
        "passage": "У мхов нет цветков, корней и проводящей системы. Размножаются мхи спорами, которые созревают в спорангиях на спорофите. В жизненном цикле, в отличие от сосудистых растений, преобладает гаплоидный (то есть с одинарным набором непарных хромосом) гаметофит (половое поколение).",
        "answer": True
    },
    {
        "question": "Диккенс женился на сестре Мэри",
        "passage": "2 апреля 1836 года Чарльз Диккенс женился на Кэтрин Томсон Хогарт (19 мая 1815 — 22 ноября 1879), старшей дочери его приятеля, журналиста Джорджа Хогарта. Диккенс был очень привязан к её сестре Мэри, ранняя смерть которой потрясла его. Другая сестра Кэтрин, Джорджина Хогарт, жила с четой Диккенсов и воспитывала их детей.",
        "answer": True
    },
    {
        "question": "Диккенс женился на Эллен Тернан",
        "passage": "В 1857 году Чарльз встретил 18-летнюю актрису Эллен Тернан и сразу влюбился. Снял для неё квартиру, долгие годы навещал свою любовь, впоследствии включил её в завещание. В 1858 году Диккенс развёлся с Кэтрин; роман с Элен продлился до смерти писателя. На сцену она больше не вышла. Этим близким отношениям посвящён художественный фильм «Невидимая женщина».",
        "answer": False
    },
    {
        "question": "Системы здравоохранения легко справляются с пандемией",
        "passage": "Пандемия опасна тем, что одновременное заболевание инфекцией множества людей может привести к перегруженности системы здравоохранения с повышенным количеством госпитализаций и летальных исходов. Системы здравоохранения могут оказаться не готовы к необычайно большому количеству тяжелобольных пациентов",
        "answer": False
    },

    {
        "question": "Чтобы производить кока-колу конртакт с компанией не нужен",
        "passage": "Компания Coca-Cola производит концентрат для кока-колы, который затем продаётся лицензированным разливщикам Coca-Cola по всему миру. Разливщики, которые имеют эксклюзивные контракты с компанией на территории, производят готовый продукт в банках и бутылках из концентрата в сочетании с фильтрованной водой и подсластителями.Типичная банка объёмом 330 мл содержит 38 граммов сахара.",
        "answer": False
    },

    {
        "question": "На официальных играх дизайн шахматных фигур может быть разным",
        "passage": "Дизайн шахматных фигур разнообразен. Стандартном де-факто стал так называемый стаунтоновский комплект, разработанный в 1849 году и по сей день применяемый в официальных шахматных турнирах, но существует множество резко отличающихся вариантов дизайна, например, фигуры, изображающие солдат конкретных армий, известных людей, сказочных персонажей, животных, персонажей определённых произведений и так далее.",
        "answer": False
    }
]
test_dataset = make_dataset(data_samples, tokenizer)
y_pred = predict_with_model(model, test_dataset)

In [ ]:
y_pred

[1, 1, 1, 1, 0, 1, 1, 1, 0, 1]

Как видно, модель хорошо справляется с вопросами, которые дублируют слова из отрывка или являются очень близкой переформулировкой. Интересно, что модель, видимо, определила, что Мэри - сестра Кэтрин Томсон Хогарт и Диккенс женился именно на последней (пример 6). Однако, есть проблемы даже с переформулировками (в отличие - отличаются (пример 5)), модель также, видимо, сбивает наличие в отрывке слов, близких по смыслу к словам из вопросов (влюбился, развелся - женился в примере 7, разнообразен - разный в примере 10). В целом, получилось 6 верных предсказаний из 10, что лучше, чем у модели, обученной с помощью Trainer.